In [1]:
import tkinter as tk
import socket
import os
from datetime import datetime
HOST = "192.168.30.228"   # 改成你的 Ingest IP
PORT = 32108             # 改成你的 Port


In [16]:
date_folder = datetime.today().strftime("%m.%d.%Y")
date_prefix = datetime.today().strftime("%m%d")  

In [17]:
SAVE_BASE = r"E:\\"
folder_path = os.path.join(SAVE_BASE, date_folder)

In [18]:
folder_path = os.path.join(SAVE_BASE, date_folder)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    log(f"📁 建立新資料夾：{folder_path}")
else:
    log(f"📁 使用現有資料夾：{folder_path}")

📁 建立新資料夾：E:\\06.26.2025


In [20]:
final_filename = f"{date_prefix}_中文"  # 0626_使用者輸入
full_path = os.path.join(date_folder, final_filename)  # 不要含 E:\ 以免多餘

In [6]:
import socket
def connect_socket():
    HOST = "192.168.30.228"
    PORT = 32108
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(3)
        log(f"🚀 嘗試連線 {HOST}:{PORT}...")
        s.connect((HOST, PORT))
        log("✅ 連線成功")
        return s  # 傳回已連線的 socket 物件
    except Exception as e:
        log("❌ 無法連線:", e)
        return None
def send_command(sock, cmd):
    try:
        sock.sendall((cmd + "\r\n").encode("cp950", errors="replace"))
        response = sock.recv(2048).decode("cp950", errors="replace")
        log(f"⬅️ Response:\n{response}")
        return response.strip()
    except Exception as e:
        log("❌ 傳送/接收失敗:", e)
        return ""
def list_encoders():
    response = send_command("List")
    if not response:
        log("⚠️ 沒收到任何回應")
        return []

    lines = response.splitlines()
    encoder_names = []
    for line in lines:
        line = line.strip()
        if line and "Mode:" in line:
            # 例如：Bak4-1   Mode: Quick
            encoder_name = line.split("Mode:")[0].strip()
            encoder_names.append(encoder_name)

    log(f"🎯 擷取到 Encoder 名稱：{encoder_names}")
    return encoder_names


In [ ]:
# encoder_utils.py
import socket
from utils import log
HOST = "192.168.30.228"
PORT = 32108

def connect_socket():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(3)
        s.connect((HOST, PORT))
        log("✅ 已建立 socket 連線")
        return s
    except Exception as e:
        log("❌ 連線失敗:", e)
        return None

def send_command(sock, cmd):
    try:
        encoded = (cmd + "\r\n").encode("cp950")
        sock.sendall(encoded)
        sock.settimeout(2)
        data = b""
        while True:
            try:
                chunk = sock.recv(1024)
                if not chunk:
                    break
                data += chunk
            except socket.timeout:
                break
        response = data.decode("cp950", errors="replace")
        log("⬅️ Response:\n", response)
        return response.strip()
    except Exception as e:
        log("❌ 指令傳送失敗:", e)
        return ""

def list_encoders():
    sock = connect_socket()
    if not sock:
        return []
    response = send_command(sock, "List")
    sock.close()

    encoders = []
    for line in response.splitlines():
        if "Mode:" in line:
            enc_name = line.split("Mode:")[0].strip()
            encoders.append(enc_name)
    if not encoders:
        log("⚠️ 沒有從 socket 抓到 encoder")
    return encoders


In [6]:

encoder_names = list_encoders()



[16:55:00] [INFO] ✅ 已建立 socket 連線
[16:55:02] [ 
OK: 
Bak4-1      Mode: Quick
State: Stopped
Start Time: 
   1920x1080 29.97 fps,      8 Ch. 48000 Hz
   Streaming enabled
Video Signal: Exists
Bak4-2      Mode: Quick
State: Stopped
Start Time: 
   1920x1080 29.97 fps,      8 Ch. 48000 Hz
   No stream
Video Signal: Exists

] ⬅️ Response:



In [ ]:
import socket
import json
import os
from utils import log, resource_path

ENCODER_CONFIG_FILE = "encoders.json"
ENCODER_CONFIG_PATH = resource_path(ENCODER_CONFIG_FILE)

encoder_config = {}

def connect_socket(ip, port):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(3)
        s.connect((ip, port))
        log(f"✅ 已建立 socket 連線：{ip}:{port}")
        return s
    except Exception as e:
        log(f"❌ 無法連線 {ip}:{port} - {e}")
        return None

def send_command(sock, cmd):
    try:
        encoded = (cmd + "\r\n").encode("cp950")
        sock.sendall(encoded)
        sock.settimeout(2)
        data = b""
        while True:
            try:
                chunk = sock.recv(1024)
                if not chunk:
                    break
                data += chunk
            except socket.timeout:
                break
        response = data.decode("cp950", errors="replace")
        log("⬅️ Response:\n" + response)
        return response.strip()
    except Exception as e:
        log(f"❌ 指令傳送失敗: {e}")
        return ""

def discover_encoders(ip: str, port: int):
    """
    掃描指定 IP/Port，透過 LIST 指令取得 encoder 名稱，並儲存到 encoders.json
    """
    sock = connect_socket(ip, port)
    if not sock:
        return []

    response = send_command(sock, "LIST")
    sock.close()

    names = []
    for line in response.splitlines():
        if "Mode:" in line:
            name = line.split("Mode:")[0].strip()
            if name and name not in names:
                names.append(name)

    if not names:
        log("⚠️ 沒有找到任何 encoder 名稱")
        return []

    # 載入原有設定，防止 encoders.json 為空或壞掉
    try:
        if os.path.exists(ENCODER_CONFIG_PATH):
            with open(ENCODER_CONFIG_PATH, "r", encoding="utf-8") as f:
                try:
                    config = json.load(f)
                except json.JSONDecodeError:
                    log("⚠️ encoders.json 無效，重新建立新設定")
                    config = {}
        else:
            config = {}
    except Exception as e:
        log(f"❌ 無法讀取 {ENCODER_CONFIG_PATH}: {e}")
        config = {}

    # 寫入新的名稱
    for name in names:
        config[name] = {"host": ip, "port": port}

    try:
        with open(ENCODER_CONFIG_PATH, "w", encoding="utf-8") as f:
            json.dump(config, f, indent=2, ensure_ascii=False)
        log(f"✅ 已將 {len(names)} 個 encoder 寫入 {ENCODER_CONFIG_PATH}")

        # 更新全域 encoder_config 變數
        global encoder_config
        encoder_config = config

    except Exception as e:
        log(f"❌ 寫入 config 失敗: {e}")
        return []

    return names


In [18]:
# 手動指定一台新設備的 IP/Port
discover_encoders("192.168.30.228", 32108)


[17:07:51] [INFO] ✅ 已建立 socket 連線：192.168.30.228:32108
[17:07:53] [INFO] ⬅️ Response:
 
OK: 
Bak4-1      Mode: Quick
State: Stopped
Start Time: 
   1920x1080 29.97 fps,      8 Ch. 48000 Hz
   Streaming enabled
Video Signal: Exists
Bak4-2      Mode: Quick
State: Stopped
Start Time: 
   1920x1080 29.97 fps,      8 Ch. 48000 Hz
   No stream
Video Signal: Exists


[17:07:53] [INFO] ❌ 寫入 config 失敗: Expecting value: line 1 column 1 (char 0)


[]

In [ ]:
discover_encoders("192.168.30.228", 32108)
print(list_encoders())

[17:06:32] [INFO] 📂 嘗試讀取 encoder 設定：c:\Users\coco\recoder\schedule_project\encoders.json
[17:06:32] [INFO] ❌ 讀取 encoders.json 時發生錯誤：Expecting value: line 1 column 1 (char 0)


ImportError: cannot import name 'discover_encoders' from 'encoder_utils' (c:\Users\coco\recoder\schedule_project\encoder_utils.py)